### Simple wave reflection model
#### Based on Brad Story's dissertation (1995)

In [7]:
# Parameters
fs = 44100 # Sampling frequency
dur = 0.5 # Duration in seconds
samples = Int(fs * dur) # Total number of samples
A = [3, 2, 1, 1, 2, 3] # Areas
# L = repeat([0.396825], length(A)) # Lengths
sections = length(A)
F = repeat([0.0], sections)
B = repeat([0.0], sections)
ρ = 0.00114 # units? g/cm^3
c = 35000 # units? cm/s
generic_loss = 0.999 # Generic loss factor

0.999

In [8]:
F[1] = 1.0 # Forward wave component init
for samp in 1:10
    println("SAMPLE ", samp)
    println("--------------------")
    println("EVENS")
    # EVEN JUNCTIONS
    for i in 1:2:sections-2
        i1 = i + 1
        i2 = i + 2
        println("junction ", i1, "-", i2, " ($i1)")
        Z = ρ * c / A[i] # Acoustic Impedance
        R1 = (A[i1] - A[i2]) / (A[i1] + A[i2]) # Reflection coefficients
        R2 = (A[i2] - A[i1]) / (A[i1] + A[i2])
        Ψ = F[i1] * R1 + B[i2] * R2
        B[i1] = (B[i2] + Ψ) * generic_loss
        F[i2] = (F[i1] + Ψ) * generic_loss
        println("Forward Pressures: ", F)
        println("Backward Pressures: ", B)
    end
    println("ODDS")
    # ODD JUNCTIONS
    for i in 1:2:sections
        i1 = i
        i2 = i + 1
        println("junction ", i1, "-", i2, " ($i1)")
        Z = ρ * c / A[i] # Acoustic Impedance
        R1 = (A[i1] - A[i2]) / (A[i1] + A[i2]) # Reflection coefficients
        R2 = (A[i2] - A[i1]) / (A[i1] + A[i2])
        Ψ = F[i1] * R1 + B[i2] * R2
        B[i1] = (B[i2] + Ψ) * generic_loss
        F[i2] = (F[i1] + Ψ) * generic_loss
        println("Forward Pressures: ", F)
        println("Backward Pressures: ", B)
        # if i == 1
        #     F[i] = B[i]
        # end
        if i2 == sections
            B[i2] = F[i2]
        end
    end
end

SAMPLE 1
--------------------
EVENS
junction 2-3 (2)
Forward Pressures: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Backward Pressures: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
junction 4-5 (4)
Forward Pressures: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Backward Pressures: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ODDS
junction 1-2 (1)
Forward Pressures: [1.0, 1.1987999999999999, 0.0, 0.0, 0.0, 0.0]
Backward Pressures: [0.1998, 0.0, 0.0, 0.0, 0.0, 0.0]
junction 3-4 (3)
Forward Pressures: [1.0, 1.1987999999999999, 0.0, 0.0, 0.0, 0.0]
Backward Pressures: [0.1998, 0.0, 0.0, 0.0, 0.0, 0.0]
junction 5-6 (5)
Forward Pressures: [1.0, 1.1987999999999999, 0.0, 0.0, 0.0, 0.0]
Backward Pressures: [0.1998, 0.0, 0.0, 0.0, 0.0, 0.0]
SAMPLE 2
--------------------
EVENS
junction 2-3 (2)
Forward Pressures: [1.0, 1.1987999999999999, 1.5968015999999998, 0.0, 0.0, 0.0]
Backward Pressures: [0.1998, 0.39920039999999996, 0.0, 0.0, 0.0, 0.0]
junction 4-5 (4)
Forward Pressures: [1.0, 1.1987999999999999, 1.5968015999999998, 0.0, 0.0, 0.0]
Backward Pressu